In [ ]:
import pandas as pd
import numpy as np # Retained as it was in the original, though not used in this specific snippet
import glob, re, os # Retained as it was in the original, though not used in this specific snippet
import PyPDF2
import fitz

def create_invoice_dictionaries(df_invoice: pd.DataFrame) -> tuple[dict, dict, dict]:
    """
    Processes an invoice DataFrame to create three dictionaries.

    Args:
        df_invoice: Pandas DataFrame containing invoice data.
                    Expected columns: 'invoice', 'invoice_name', 'company_code', 'invoice_date'.

    Returns:
        A tuple containing three dictionaries:
        1. dict_invoice_number_to_name: Maps invoice number (str) to invoice name.
        2. dict_invoice_number_to_company_code: Maps invoice number (str) to company code.
        3. dict_invoice_number_to_date: Maps invoice number (str) to invoice date.
    """
    # Ensure 'invoice' column is treated as string for dictionary keys
    # It's good practice to handle potential missing values if necessary, e.g., .fillna('')
    list_invoice_numbr = df_invoice['invoice'].astype(str).tolist()
    list_invoice_name = df_invoice['invoice_name'].tolist()
    list_company_code = df_invoice['company_code'].tolist()
    #list_invoice_date = df_invoice['invoice_date'].tolist() # Assuming dates are already in desired format

    # Process 'invoice_date' column:
    # 1. Convert to datetime objects. 'errors="coerce"' will turn unparseable dates into NaT (Not a Time).
    # 2. Format the datetime objects to 'YYYYMMDD' string format.
    # 3. Fill any NaT values (resulting from unparseable dates) with an empty string.
    # 4. Convert the Series to a list.
    list_invoice_date = pd.to_datetime(df_invoice['invoice_date'], errors='coerce') \
                          .dt.strftime('%Y%m%d') \
                          .fillna('') \
                          .tolist()

    # Create dictionaries using dictionary comprehension for conciseness and efficiency
    # This is equivalent to zipping the lists and then converting to a dictionary

    # Dictionary 1: Maps invoice number to invoice name Example: {'123': 'Invoice Alpha', '456': 'Invoice Beta'}
    dict_invoice_number_to_name = {
        num: name for num, name in zip(list_invoice_numbr, list_invoice_name)
    }

    # Dictionary 2: Maps invoice number to company code Example: {'123': 'CC001', '456': 'CC002'}
    dict_invoice_number_to_company_code = {
        num: code for num, code in zip(list_invoice_numbr, list_company_code)
    }

    # Dictionary 3: Maps invoice number to invoice date Example: {'123': '2023-01-15', '456': '2023-01-20'}
    dict_invoice_number_to_date = {
        num: date for num, date in zip(list_invoice_numbr, list_invoice_date)
    }

    return dict_invoice_number_to_name, dict_invoice_number_to_company_code, dict_invoice_number_to_date


def format_dictionary_keys_in_place(dictionary_invoice):
        # Iterate over a copy of the keys to avoid RuntimeError
        for key in list(dictionary_invoice.keys()):
            new_key = key.replace(':', '_').replace('/', '_')
            if new_key != key:  # Only update if the key has changed
                dictionary_invoice[new_key] = dictionary_invoice.pop(key)
        return dictionary_invoice  # Return the modified dictionary

#def rename_pdf_files
def rename_pdf_files(directory_invoice, list_invoice_numbr, dict_invoice_name_number, dict_invoice_name_company_code, formatted_dates_dict):
    pdf_rename =0
    for filename in os.listdir(directory_invoice):
            # Check if the file is a PDF and its name (without extension) is in list_invoice_name
            if (filename.endswith(".pdf")): 
                # Get the invoice num (filename without extension)
                invoice_numbr = os.path.splitext(filename)[0]
                # Check if the filename contains '_merged'
                if '_merged' in invoice_numbr:
                    # Remove '_merged' from the filename
                    invoice_numbr = invoice_numbr.replace('_merged', '')
                    if invoice_numbr in list_invoice_numbr:                        
                
                        # Get the corresponding value from dict_invoice_name_number
                        value = dict_invoice_name_number.get(str(invoice_numbr), "")  
                        invoice_date = formatted_dates_dict.get(str(invoice_numbr), "")
                        company_code = dict_invoice_name_company_code.get(str(invoice_numbr), "")                
            
                        new_filename = f"{value}_{invoice_numbr}_{company_code}_{invoice_date}.pdf" # Construct the new filename
                        # Get the full paths for the old and new filenames
                        old_file_path = os.path.join(directory_invoice, filename)
                        new_file_path = os.path.join(directory_invoice, new_filename)

                        try:
                            os.rename(old_file_path, new_file_path) # Rename the file         
                            print(f"Renamed: {filename} -> {new_filename}")
                            pdf_rename +=1
                        except Exception as e:
                            print(f"failed to rename {filename}: {e}")
                            continue
                    print(f'done rename {pdf_rename} pdf files')
    return pdf_rename

page_images = []
page_text = []
def find_text_in_pdf(path_pdf, file_pdf):
    """
    Removes blank pages and the first page if it's blank from a PDF file.
    Args:
        path_pdf (str): Path to the directory containing the PDF file.
        file_pdf (str): Name of the PDF file.
    Returns:
        None
    """
    with open(path_pdf + file_pdf, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        pdf_writer = PyPDF2.PdfWriter()

        # Check the first page
        if pdf_reader.pages:  # Ensure there are pages
            first_page = pdf_reader.pages[0]
            first_page_text = first_page.extract_text()
            if first_page_text.strip() == "":
                # First page is blank, skip it
                start_page = 1
            else:
                start_page = 0
        else:
            return # no pages to process

        # Process remaining pages
        for page_num in range(start_page, len(pdf_reader.pages)): #len(pdf_reader.pages)
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                if text.strip() != "":
                    #pdf_writer.add_page(page)
                    page_text.append(page_num)
                    #print(f"Page {page_num} has text: {text}")
    #print(len(page_text))
    print(f"total text pages: {len(pdf_reader.pages)}")
    return page_text

def find_images_in_pdf(path_pdf, file_pdf):
    pdf = fitz.Document(path_pdf + file_pdf)
    #page_images = []
    for i in range(len(pdf)):
        images = pdf.get_page_images(i)
        has_image = False if len(images) == 0 else True
        # print(f"{i}: {hasimage}: {images}")
        if has_image:
            page_images.append(i)
            #print(f"Page {i} has images: {images}")
    print(f"total image pages: {len(pdf)}")
    return page_images
    
        # Write the new PDF
        #with open(path_pdf + "no_blank_" + file_pdf, 'wb') as output_file:
                #pdf_writer.write(output_file)


# --- Example Usage (similar to your original script) ---
if __name__ == "__main__":
    username = 'john.tan' # Replace with actual username or make it dynamic
    excel_file_path = rf"C:/Users/{username}/Downloads/esker_merged/invoice_rename.xlsx"
    directory_invoice = rf"C:/Users/{username}/Downloads/esker_merged/" # Not used in this refactored function
    pdf_files = [f for f in os.listdir(directory_invoice) if (f.lower().endswith('.pdf'))] #[pdf_files[0]]


    try:
        for pdf_file in pdf_files:
            print(pdf_file)
            try:
                find_text_in_pdf(directory_invoice, pdf_file)
            except Exception as e:
                print(e)
                break
            try:
                find_images_in_pdf(directory_invoice, pdf_file)
            except Exception as e:
                print(e)
                break
            list_pages_text_images = list(set(page_images + page_text))
            with open(directory_invoice + pdf_file, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                pdf_writer = PyPDF2.PdfWriter()

                for page_num in list_pages_text_images:
                    try:

                        page = pdf_reader.pages[page_num]
                        pdf_writer.add_page(page)
                    except Exception as e:  
                        print(e)
                        break
                with open(directory_invoice + "" + pdf_file, 'wb') as output_file:
                    pdf_writer.write(output_file) #write the new pdf #no_blank
                    print(f"Processed {pdf_file}: removed blank pages.")
    except Exception as e:
            print(f"An error occurred while processing the PDF files: {e}")


    try:
        # Load the DataFrame
        df_invoice = pd.read_excel(excel_file_path, sheet_name='invoice_rename', engine='openpyxl')
        list_invoice_numbr = df_invoice['invoice'].astype(str).tolist()

        # Call the refactored function to get the dictionaries
        invoice_to_name, invoice_to_code, invoice_to_date = create_invoice_dictionaries(df_invoice.copy()) # Pass a copy to avoid SettingWithCopyWarning if df_invoice is modified later

        format_invoice_to_name = format_dictionary_keys_in_place(invoice_to_name)
        format_invoice_to_code = format_dictionary_keys_in_place(invoice_to_code)
        format_invoice_to_date = format_dictionary_keys_in_place(invoice_to_date)
        
        from pandas import Timestamp # Assuming Timestamp is from pandas #2

        format_invoice_to_code = {k: v for k, v in format_invoice_to_code.items() if k in format_invoice_to_name}

        format_invoice_to_date = {k: v for k, v in format_invoice_to_date.items() if k in format_invoice_to_name}

        rename_pdf_files(directory_invoice, list_invoice_numbr, format_invoice_to_name, format_invoice_to_code, format_invoice_to_date)

        # Print the dictionaries to verify (optional)
        print("--- Invoice Number to Name ---")
        for inv_num, name in list(format_invoice_to_name.items())[:3]: # Print first 3 for brevity
            print(f"{inv_num}: {name}")

        print("\n--- Invoice Number to Company Code ---")
        for inv_num, code in list(format_invoice_to_code.items())[:3]: # Print first 3 for brevity
            print(f"{inv_num}: {code}")

        print("\n--- Invoice Number to Invoice Date ---")
        for inv_num, date in list(format_invoice_to_date.items())[:3]: # Print first 3 for brevity
            print(f"{inv_num}: {date}")

    except FileNotFoundError:
        print(f"Error: The file '{excel_file_path}' was not found.")
    except KeyError as e:
        print(f"Error: A required column is missing from the Excel file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")



IRS34547_merged.pdf
total text pages: 27
total image pages: 27
Processed IRS34547_merged.pdf: removed blank pages.
RS25040592_merged.pdf
total text pages: 12
total image pages: 12
sequence index out of range
Processed RS25040592_merged.pdf: removed blank pages.
Renamed: IRS34547_merged.pdf -> ISLAND RECOVERY SERVICES PTE LTD_IRS34547_SG77_20250508.pdf
done rename 1 pdf files
done rename 1 pdf files
done rename 1 pdf files
Renamed: RS25040592_merged.pdf -> RESOLUTE SOLUTIONS PTE LTD_RS25040592_SG80_20250411.pdf
done rename 2 pdf files
--- Invoice Number to Name ---
IRS34547: ISLAND RECOVERY SERVICES PTE LTD
RS25040592: RESOLUTE SOLUTIONS PTE LTD

--- Invoice Number to Company Code ---
IRS34547: SG77
RS25040592: SG80

--- Invoice Number to Invoice Date ---
IRS34547: 20250508
RS25040592: 20250411
